In [2]:
import pandas as pd
import numpy as np

#### Ouverture du csv des annotations, changement des colonnes

In [3]:
samplerate = 16000

directory = './challenge-m2-sid/'
dir_audio = directory + 'corpus/audio/'
dir_texte = directory + 'corpus/text/'
dir_video = directory + 'corpus/video/'

audio_extension = "_mono.wav"
texte_extension = ".xml"
video_extension = ".m4v"

annot = pd.read_csv(directory+'annotations_challenge_sid.csv', sep="\t", index_col=0, header=0).replace(-1,np.nan)
noteurs = annot.columns[1:]

annot['doc']      = ["_".join(e.split("_")[0:2]) for e in annot["code_doc"]]
annot['audio']    = [int(e.split("_")[2][0])     for e in annot["code_doc"]]
annot['video']    = [int(e.split("_")[2][1])     for e in annot["code_doc"]]
annot['texte']    = [int(e.split("_")[2][2])     for e in annot["code_doc"]]
annot['isGraded'] = [int(e.split("_")[3])        for e in annot["code_doc"]]

annot = annot.drop("code_doc", axis=1)
annot.head(6)

,il08_09,vg04_05,fd03_04,la09_10,cg13_14,mb00_12,doc,audio,video,texte,isGraded
1,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,1,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,1,1,0
4,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,0,1,0
5,NaN,NaN,NaN,NaN,NaN,NaN,119_8,0,0,1,0
6,NaN,NaN,NaN,NaN,NaN,76.0,57_6,1,0,0,1


#### Selection des audio notés et calcul de la moyenne

In [4]:
audio_only = annot[(annot['isGraded'] == 1) &
                   (annot['audio']    == 1) &
                   (annot['video']    == 0) &
                   (annot['texte']    == 0)
                  ].copy()

audio = annot[(annot['isGraded'] == 1) & 
              (annot['audio']    == 1)
             ].copy()

audio['moyenne']      = audio[noteurs].apply(np.nanmean, axis=1)
audio_only['moyenne'] = audio_only[noteurs].apply(np.nanmean, axis=1)

print("audio :",      len(set(audio['doc'])))
print("audio_only :", len(set(audio_only['doc'])))

audio_only[['doc','moyenne']].groupby(['doc']).agg(['count','mean']).head(7)

audio : 55
audio_only : 33


moyenne      
        count  mean
doc                
100_1       1  47.5
107_7       1  76.0
108_3       1  66.0
128_5       1  81.0
13_2        1  62.5
147_1       1  64.0
148_4       1  13.0